In [2]:
from src.models.model1.model_preprocess import get_data

In [3]:
import pandas as pd
g = pd.read_pickle ('data/interim/X_group.pkl')
t = pd.read_pickle ('data/interim/X_technique.pkl')
y = pd.read_pickle ('data/interim/y_cleaned.pkl')
tactics = pd.read_csv ('data/raw/tactics_order.csv', index_col= 0)

In [4]:
y['label'].value_counts()

label
0.0    49716
1.0     3052
Name: count, dtype: int64

- limit technique based on earliest stage

In [5]:
technique_earliest_stage = pd.merge (
    left = t[['technique_ID', 'input_technique_tactics']].explode ('input_technique_tactics'),
    right = tactics,
    how = 'left', left_on= 'input_technique_tactics', right_on= 'tactic_name'
)
technique_earliest_stage = technique_earliest_stage.groupby ('technique_ID', as_index= False).agg(min)
technique_earliest_stage.drop(columns = ['input_technique_tactics', 'tactic_name'], inplace= True)
technique_earliest_stage.rename (columns= {'stage_order': 'technique_earliest_stage'}, inplace= True)

In [6]:
group_earliest_stage = pd.merge (
    left = y[y['label']==1], 
    right = technique_earliest_stage,
    on = 'technique_ID', how = 'left'
)
group_earliest_stage = group_earliest_stage[['group_ID', 'technique_earliest_stage']].groupby ('group_ID', as_index= False).agg(min)
group_earliest_stage.rename (columns= {'technique_earliest_stage': 'group_earliest_stage'}, inplace= True)
group_earliest_stage


,group_ID,group_earliest_stage
0,G0001,0
1,G0002,5
2,G0003,0
3,G0004,0
4,G0005,1
...,...,...
128,G1009,0
129,G1011,0
130,G1012,0
131,G1013,0


In [7]:
group_technique_interaction_matrix = pd.merge (
    left = y,
    right = technique_earliest_stage,
    how = 'left', on = 'technique_ID'
)
group_technique_interaction_matrix = pd.merge (
    left = group_technique_interaction_matrix, 
    right = group_earliest_stage, 
    how = 'left', on = 'group_ID'
)
group_technique_interaction_matrix = group_technique_interaction_matrix [group_technique_interaction_matrix ['group_earliest_stage'] <= group_technique_interaction_matrix ['technique_earliest_stage']]
group_technique_interaction_matrix.drop(columns= ['group_earliest_stage', 'technique_earliest_stage', 'tactic_ID'], inplace= True)
group_technique_interaction_matrix

,group_ID,technique_ID,label
0,G0099,T1548.002,0.0
1,G0099,T1134,0.0
2,G0099,T1134.002,0.0
3,G0099,T1134.001,0.0
4,G0099,T1531,0.0
...,...,...,...
52763,G0045,T1102.002,0.0
52764,G0045,T1102.001,0.0
52765,G0045,T1102.003,0.0
52766,G0045,T1047,1.0


In [8]:
group_technique_interaction_matrix['label'].value_counts()

label
0.0    44474
1.0     3052
Name: count, dtype: int64

In [10]:
reduced_labels = pd.read_pickle ('scripts/tmp_reduced_labels.pkl')

In [12]:
reduced_labels['label'].value_counts()

label
0.0    44474
1.0     3052
Name: count, dtype: int64

- limit group based on number of positive instances